In [1]:
import PyPDF2
import unidecode
import pandas as pd
from collections import Counter
import csv
import os
import os.path
import random
import re
import datetime

### Create lists with all paths to all files

### Generate target feature for each report ('read' and summarised the polarity)

In [2]:
folders = ["ciclo_3",
           "ciclo_4",
           "edicoes_anteriores/sorteio_34",
           "edicoes_anteriores/sorteio_35",
           "edicoes_anteriores/sorteio_36",
           "edicoes_anteriores/sorteio_37",
           "edicoes_anteriores/sorteio_38",
           "edicoes_anteriores/sorteio_39",
           "edicoes_anteriores/sorteio_40"]

seq_folders = []
file_names = []
file_names_and_paths = []

for folder in folders:
    directory = '../programa_de_fiscalizacao_em_entes_federativos/' + folder
    
    number_of_files = len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])
    
    for i in range(0, number_of_files):
        file_name_and_path = directory + "/" + os.listdir(directory)[i]
        if (".pdf" in file_name_and_path):
            seq_folders.append(folder)
            file_names.append(os.listdir(directory)[i])
            file_names_and_paths.append(file_name_and_path)

file_names_and_paths[0:1]

['../programa_de_fiscalizacao_em_entes_federativos/ciclo_3/8998-Santo Antônio de Jesus-BA.pdf']

In [3]:
sentilex_database = pd.read_csv("../sentilex/99_create_initial_sentilex_database.csv",
                               sep = ";")

sentilex_database.adjective = sentilex_database.adjective.str.normalize('NFKD').\
                                str.encode('ascii', errors='ignore').str.decode('utf-8')

In [4]:
cities = pd.DataFrame()

print("List of reports read and summarised")

for file_number in range(0, len(file_names_and_paths)):
    folder = seq_folders[file_number]
    file_name = file_names[file_number]
    file_name_and_path = file_names_and_paths[file_number]
    print(str(datetime.datetime.now()) + ' ' + file_name_and_path)
    
    # read report using external library pdf miner and save in 'temp_report.txt'
    command_to_cmd = 'pdf2txt.py "' + file_name_and_path + '" > temp_report.txt'
    os.system(command_to_cmd)
    
    # read temporary file
    temporary_file = open('temp_report.txt', 'r')
    
    whole_text = ''
    
    for line in temporary_file:
        whole_text += line
    
    words = re.findall(r"[\w']+", unidecode.unidecode(re.sub('\d', ' ', whole_text).lower()))

    # create the frequencies
    words_freq = pd.DataFrame.from_dict(Counter(words), orient = 'index').reset_index()
    words_freq.columns = ['word', 'freq']
    words_freq['pct'] = words_freq['freq']/sum(words_freq.freq)

    # aggregate polarity
    words_freq_polarity = words_freq.merge(sentilex_database,
                                           left_on = "word",
                                           right_on = "adjective",
                                           how = "left").iloc[:, [0, 1, 2, 4]]
    
    words_freq_polarity_fill = words_freq_polarity.fillna(0)
    
    # summarise
    number_of_words = words_freq_polarity_fill.freq.sum()
    pct_pol_neg = words_freq_polarity_fill[words_freq_polarity_fill.polarity == -1].pct.sum()
    pct_pol_pos = words_freq_polarity_fill[words_freq_polarity_fill.polarity == 1].pct.sum()
    pct_pol_neu = words_freq_polarity_fill[words_freq_polarity_fill.polarity == 0].pct.sum()
    
    current_city = pd.DataFrame({"folder": folder,
                                 "file_name": file_name,
                                 "number_of_words": number_of_words,
                                 "pct_pol_neg": pct_pol_neg,
                                 "pct_pol_pos": pct_pol_pos,
                                 "pct_pol_neu": pct_pol_neu},
                                index = [0])
    
    cities = cities.append(current_city)

List of reports read and summarised
2019-05-18 10:08:59.859956 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_3/8998-Santo Antônio de Jesus-BA.pdf
2019-05-18 10:09:36.854276 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_3/9024-Ulianópolis-PA.pdf
2019-05-18 10:10:22.367380 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_3/9010-Aldeias Altas-MA.pdf
2019-05-18 10:11:09.559848 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_3/9034-Paraíba do Sul-RJ.pdf
2019-05-18 10:11:20.214852 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_3/9045-Governador Celso Ramos-SC.pdf
2019-05-18 10:11:22.857141 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_3/9016-Pirajuba-MG.pdf
2019-05-18 10:11:29.294191 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_3/9018-Naviraí-MS.pdf
2019-05-18 10:11:52.111644 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_3/9015-Nova Lima-MG.pdf
2019-05-18 10:12:05.583004 ../programa_de_fiscalizacao_em_entes_federativos/ci

2019-05-18 10:31:17.823804 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10323-São Borja-RS.pdf
2019-05-18 10:31:23.284250 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10374-Jataí-GO.pdf
2019-05-18 10:31:53.508595 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10379-São Domingos-SC.pdf
2019-05-18 10:32:23.549326 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10407-Oeiras-PI.pdf
2019-05-18 10:32:59.494470 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10291-Criciúma-SC.pdf
2019-05-18 10:33:08.379807 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10383-Caldas Novas-GO.pdf
2019-05-18 10:33:26.059473 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10311-Manhuaçu-MG.pdf
2019-05-18 10:33:39.408910 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10289-Estância-SE.pdf
2019-05-18 10:34:10.442594 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10403-Tobias Barreto-SE.pdf
2019-05-18 10:34:56.108323 ../

2019-05-18 11:07:41.912722 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10388-Sertãozinho-SP.pdf
2019-05-18 11:07:49.411767 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10442-Viçosa-MG.pdf
2019-05-18 11:07:53.723665 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10286-Cascavel-PR.pdf
2019-05-18 11:08:23.599128 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10438-Icó-CE.pdf
2019-05-18 11:08:33.266171 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10287-Londrina-PR.pdf
2019-05-18 11:08:47.021261 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10312-Juiz de Fora-MG.pdf
2019-05-18 11:09:04.983501 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10324-Santa Rosa-RS.pdf
2019-05-18 11:09:12.593932 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10361-Rio Verde-GO.pdf
2019-05-18 11:09:40.966323 ../programa_de_fiscalizacao_em_entes_federativos/ciclo_4/10371-Aquidauana-MS.pdf
2019-05-18 11:10:20.480075 ../programa

2019-05-18 11:20:42.808574 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_34/1827-Urucurituba-AM.pdf
2019-05-18 11:20:58.767777 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_34/1830-Canápolis-BA.pdf
2019-05-18 11:21:11.907742 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_34/1859-Quipapá-PE.pdf
2019-05-18 11:22:04.730123 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_35/1897-Palminópolis-GO.pdf
2019-05-18 11:22:09.097951 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_35/1896-Diorama-GO.pdf
2019-05-18 11:22:11.309723 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_35/1916-Agrestina-PE.pdf
2019-05-18 11:22:34.440891 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_35/1920-Santo Antônio dos Milagres-PI.pdf
2019-05-18 11:22:40.461666 ../programa_de_fiscalizacao_em_entes_federativos/edicoe

2019-05-18 11:31:20.656567 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_36/2459-Bela Vista da Caroba-PR.pdf
2019-05-18 11:31:22.883507 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_36/2491-Sobral-CE.pdf
2019-05-18 11:31:37.971645 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_36/2477-Terra Rica-PR.pdf
2019-05-18 11:31:51.677593 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_36/2489-Japoatã-SE.pdf
2019-05-18 11:32:09.524395 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_36/2483-Pontal do Paraná-PR.pdf
2019-05-18 11:32:27.293264 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_36/2497-Santo André-PB.pdf
2019-05-18 11:32:30.392551 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_36/2487-Groaíras-CE.pdf
2019-05-18 11:32:41.589692 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_a

2019-05-18 11:43:32.863260 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_37/2851-Laranjeiras do Sul-PR.pdf
2019-05-18 11:43:44.177743 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_37/2805-Araguanã-MA.pdf
2019-05-18 11:44:01.071150 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_37/2773-Roteiro-AL.pdf
2019-05-18 11:44:22.020815 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_37/2765-Uraí-PR.pdf
2019-05-18 11:44:28.033622 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_37/2775-Flor do Sertão-SC.pdf
2019-05-18 11:44:35.554944 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_37/2819-Mirante da Serra-RO.pdf
2019-05-18 11:44:54.300574 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_37/2835-Olindina-BA.pdf
2019-05-18 11:45:02.787029 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anter

2019-05-18 11:55:12.841879 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_38/3004-Gaúcha do Norte-MT.pdf
2019-05-18 11:55:22.881703 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_38/3001-Douradina-MS.pdf
2019-05-18 11:55:27.936605 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_38/3066-Mirassolândia-SP.pdf
2019-05-18 11:55:31.777469 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_38/2997-Vargem Bonita-MG.pdf
2019-05-18 11:55:33.981227 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_38/3054-Boquim-SE.pdf
2019-05-18 11:55:40.447470 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_38/3006-Nova Canaã do Norte-MT.pdf
2019-05-18 11:55:52.129846 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_38/2969-Aracoiaba-CE.pdf
2019-05-18 11:56:16.024395 ../programa_de_fiscalizacao_em_entes_federativos/edic

2019-05-18 12:11:36.155938 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_39/3184-Camutanga-PE.pdf
2019-05-18 12:11:57.464392 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_39/3228-Borborema-SP.pdf
2019-05-18 12:12:03.825066 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_39/3230-Itaju-SP.pdf
2019-05-18 12:12:06.845606 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_39/3186-Jucati-PE.pdf
2019-05-18 12:12:30.214517 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_39/3154-Coelho Neto-MA.pdf
2019-05-18 12:13:30.422347 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_39/3126-Barra do Mendes-BA.pdf
2019-05-18 12:14:04.718904 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_39/3214-Porto Lucena-RS.pdf
2019-05-18 12:14:26.789997 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorte

2019-05-18 12:31:50.314711 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_40/3352-Lupércio-SP.pdf
2019-05-18 12:32:00.513958 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_40/3330-Curaçá-BA.pdf
2019-05-18 12:32:30.241074 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_40/3394-Porto dos Gaúchos-MT.pdf
2019-05-18 12:32:45.278968 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_40/3421-Imaculada-PB.pdf
2019-05-18 12:33:14.751771 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_40/3411-Afogados da Ingazeira-PE.pdf
2019-05-18 12:34:00.431638 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_40/3419-Maranhãozinho-MA.pdf
2019-05-18 12:35:02.089160 ../programa_de_fiscalizacao_em_entes_federativos/edicoes_anteriores/sorteio_40/3320-Rio Branco do Ivaí-PR.pdf
2019-05-18 12:35:12.029027 ../programa_de_fiscalizacao_em_entes_feder

In [6]:
os.remove("temp_report.txt")

In [7]:
cities.to_csv("../target_feature/01_create_target_feature.csv",
              sep=';',
              encoding='utf-8',
              index=False)